In [ ]:
import pandas as pd 
import io
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from google.colab import files
from datetime import date, time, datetime
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# Emergency Department Waiting Times 

### Data Upload

In [ ]:
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['CleanedData-withAge-Total.csv']))

Saving CleanedData-withAge-Total.csv to CleanedData-withAge-Total.csv


### Data Clean Up

Helper functions to clean up Dataset, split into test and train, and function calls

In [ ]:
def cleanDataset(myData):
    myData.drop(index=myData[myData['CTAS_1.0'] == 1].index, inplace=True)
    myData.drop(index=myData[myData['CTAS_2.0'] == 1].index, inplace=True)
    myData = myData.drop(["CTAS_3.0","CTAS_2.0","CTAS_1.0","CTAS_4.0","CTAS_5.0","PTN_SEX_U"],axis=1)
    myData = myData.dropna()
    for col in list(myData.columns):
        myList=[]
        myList.append(col)
        myData[myList] = myData[myList].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
    return myData

In [ ]:
def splitIntoTrainTest(myData):
    Xtrain, Xtest = train_test_split(myData, test_size=0.2)
    yTrain = Xtrain["Total_Time_Benchmark"]
    del Xtrain["Total_Time_Benchmark"]

    yTest = Xtest["Total_Time_Benchmark"]
    del Xtest["Total_Time_Benchmark"]
    return Xtrain, Xtest, yTrain, yTest

In [ ]:
myData = cleanDataset(df)
Xtrain, Xtest, yTrain, yTest = splitIntoTrainTest(myData)

### Support Vector Machine Classifiers

In [ ]:
def trainSVM(Xtrain, Xtest, yTrain, yTest, k):
    clf = svm.SVC(kernel=k,random_state=0,class_weight='balanced')
    clf.fit(Xtrain,yTrain)
    y_pred = clf.predict(Xtest)
    print("Accuracy:", accuracy_score(yTest, y_pred))
    print("Recall:", precision_score(yTest, y_pred))
    print("Specificity:", recall_score(yTest, y_pred))

In [ ]:
trainSVM(Xtrain, Xtest, yTrain, yTest, 'linear')

Done Training
Accuracy: 0.6670949780490416
Recall: 0.5825786977145321
Specificity: 0.6972903225806452


In [ ]:
trainSVM(Xtrain, Xtest, yTrain, yTest, 'poly')

Done Training
Accuracy: 0.7141021522646964
Recall: 0.642806352216165
Specificity: 0.6998709677419355


In [ ]:
trainSVM(Xtrain, Xtest, yTrain, yTest, 'rbf')

Done Training
Accuracy: 0.7120676731984152
Recall: 0.642274472168906
Specificity: 0.6908387096774193


### Random Forest Classifier

In [ ]:
def trainRF(Xtrain, Xtest, yTrain, yTest):
    clf=RandomForestClassifier(random_state = 42,class_weight='balanced')#class_weight='balanced'
    clf.fit(Xtrain,yTrain)
    y_pred = clf.predict(Xtest)
    print("Accuracy:", accuracy_score(yTest, y_pred))
    print("Recall:", precision_score(yTest, y_pred))
    print("Specificity:", recall_score(yTest, y_pred))

In [ ]:
trainRF(Xtrain, Xtest, yTrain, yTest)

Done Training
Accuracy: 0.7230966912945711
Recall: 0.7069020866773675
Specificity: 0.568258064516129


### Decision Tree For Feature Selection

In [ ]:
def trainDT(Xtrain, Xtest, yTrain, yTest):
    clf=DecisionTreeClassifier(class_weight='balanced')
    clf.fit(Xtrain,yTrain)
    y_pred = clf.predict(Xtest)
    print("Accuracy:", accuracy_score(yTest, y_pred))
    print("Recall:", precision_score(yTest, y_pred))
    print("Specificity:", recall_score(yTest, y_pred))
    myList=[]
    myList.extend(Xtrain.columns)
    print(myList)
    coefs_with_fns = sorted(zip(clf.feature_importances_, myList))
    df = pd.DataFrame(coefs_with_fns)
    df.columns = 'Coefficient', 'Feature'
    df.sort_values(by='coefficient')
    print(df)

In [ ]:
trainDT(Xtrain, Xtest, yTrain, yTest)

Done Training
Accuracy: 0.6376485705107613
Recall: 0.5630619059851014
Specificity: 0.5656774193548387
['ARRIVAL_MODE_Air Ambulance', 'ARRIVAL_MODE_Ground Ambulance', 'ARRIVAL_MODE_No Ambulance', 'INIT_TREAT_LOC_GRP_Intake', 'INIT_TREAT_LOC_GRP_MET', 'INIT_TREAT_LOC_GRP_Main', 'INIT_LOC_GRP_Intake', 'INIT_LOC_GRP_MET', 'INIT_LOC_GRP_Main', 'INIT_LOC_GRP_WR Intake', 'INIT_LOC_GRP_WR MET', 'INIT_LOC_GRP_WR Main', 'INIT_LOC_GRP_WR Non ED', 'Number_of_Patients_In_Waiting_Before', 'Number_of_Patients_In_Waiting_Before_CTAS', 'Number_of_Patients_In_Waiting_After', 'Number_of_Patients_In_Waiting_After_CTAS', 'Number_of_Patients_Waiting_For_Discharge', 'Number_of_Patients_Waiting_For_Discharge_CTAS', 'Number_of_Patients_Waiting_For_Admission', 'Number_of_Patients_Waiting_For_Admission_CTAS', 'Number_of_Physicians_At_Arrival', 'Number_of_Physicians_At_3Hours_After', 'Number_of_Physicians_At_2Hours_After', 'Number_of_Physicians_At_1Hours_After', 'Number_of_Physicians_At_3Hours_Before', 'Number_of

### Artifical Neural Network

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Dense
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import Adam
from sklearn.utils import class_weight
from sklearn.model_selection import cross_validate
import tensorflow as tf
from sklearn.metrics import confusion_matrix,f1_score, precision_score, recall_score

In [ ]:
def buildModel(dim):
    model = Sequential()
    model.add(Dense(32, input_dim=dim, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    return model

In [ ]:
def fitModel(myData, model, Xtrain, yTrain, Xtest, yTest):
  opt = Adam(lr=0.0001)
  class_weight_dict = dict(enumerate(class_weight.compute_class_weight('balanced', np.unique(yTrain), yTrain)))
  model.compile(loss='mse', optimizer=opt, metrics=["accuracy",tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
  callbacks = [
            EarlyStopping(patience=100, verbose=1),
            ReduceLROnPlateau(factor=0.50, patience=25, min_lr=0.0001, verbose=1),
            ModelCheckpoint(
                filepath="Test1.h5",
                verbose=1, save_best_only=True, save_weights_only=True, monitor="val_loss", mode='min')]
  model.fit(Xtrain, yTrain, epochs=150, batch_size=50,validation_split=0.20,verbose=1,shuffle=True,callbacks=callbacks,class_weight=class_weight_dict)
  #y_pred = model.predict(Xtest)
  """ scoring = ['accuracy', 'precision', 'recall', 'roc_auc']
  scores = cross_validate(model, Xtrain,yTrain, cv =5, scoring=scoring, return_train_score=False)
  print(sum(scores["test_accuracy"])/len(scores["test_accuracy"]))
  print(sum(scores["test_precision"])/len(scores["test_precision"]))
  print(sum(scores["test_recall"])/len(scores["test_recall"]))"""
  y_pred = model.predict_classes(Xtest, verbose=0)
  print("Accuracy:", accuracy_score(yTest, y_pred))
  print("Recall:", precision_score(yTest, y_pred))
  print("Specificity:", recall_score(yTest, y_pred))

In [ ]:
model = buildModel(49)
fitModel(myData, model, Xtrain, yTrain, Xtest, yTest)

Epoch 1/150
598/598 [==============================] - 3s 3ms/step - loss: 0.2495 - accuracy: 0.4078 - precision_5: 0.4069 - recall_5: 0.9979 - val_loss: 0.2506 - val_accuracy: 0.4629 - val_precision_5: 0.4359 - val_recall_5: 0.9377

Epoch 00001: val_loss improved from inf to 0.25065, saving model to Test1.h5
Epoch 2/150
598/598 [==============================] - 1s 2ms/step - loss: 0.2465 - accuracy: 0.4726 - precision_5: 0.4329 - recall_5: 0.9019 - val_loss: 0.2472 - val_accuracy: 0.5689 - val_precision_5: 0.4923 - val_recall_5: 0.7606

Epoch 00002: val_loss improved from 0.25065 to 0.24722, saving model to Test1.h5
Epoch 3/150
598/598 [==============================] - 1s 2ms/step - loss: 0.2418 - accuracy: 0.5694 - precision_5: 0.4840 - recall_5: 0.7592 - val_loss: 0.2423 - val_accuracy: 0.6014 - val_precision_5: 0.5199 - val_recall_5: 0.6977

Epoch 00003: val_loss improved from 0.24722 to 0.24229, saving model to Test1.h5
Epoch 4/150
598/598 [==============================] - 1s 2

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Accuracy: 0.7006103437198844
Recall: 0.6162764771460424
Specificity: 0.7199791612399062


#### Feature Reduction for ANN Classifier

Following Model Has 42 features, reduced from 49 earlier

In [ ]:
myData_reduced_42 = myData.drop(["ARRIVAL_MODE_Air Ambulance",'PTN_AGE_>100',"Labs_Ordered_Hour_After_Arrival",
                          "Number_of_Patients_In_Waiting_Before_CTAS","Number_of_Patients_In_Waiting_Before",
                          "PTN_AGE_1-10","INIT_LOC_GRP_WR Non ED"],axis=1)
Xtrain_42, Xtest_42, yTrain_42, yTest_42 = splitIntoTrainTest(myData_reduced_42)


In [ ]:
model_42 = buildModel(42)
fitModel(myData_reduced_42, model_42,  Xtrain_42, yTrain_42, Xtest_42, yTest_42)

Epoch 1/150
598/598 [==============================] - 3s 3ms/step - loss: 0.2505 - accuracy: 0.4227 - precision_6: 0.4131 - recall_6: 0.9635 - val_loss: 0.2495 - val_accuracy: 0.5201 - val_precision_6: 0.4586 - val_recall_6: 0.7964

Epoch 00001: val_loss improved from inf to 0.24951, saving model to Test1.h5
Epoch 2/150
598/598 [==============================] - 1s 2ms/step - loss: 0.2450 - accuracy: 0.5456 - precision_6: 0.4702 - recall_6: 0.7556 - val_loss: 0.2398 - val_accuracy: 0.6046 - val_precision_6: 0.5247 - val_recall_6: 0.6091

Epoch 00002: val_loss improved from 0.24951 to 0.23984, saving model to Test1.h5
Epoch 3/150
598/598 [==============================] - 1s 2ms/step - loss: 0.2360 - accuracy: 0.6030 - precision_6: 0.5107 - recall_6: 0.6153 - val_loss: 0.2291 - val_accuracy: 0.6185 - val_precision_6: 0.5391 - val_recall_6: 0.6254

Epoch 00003: val_loss improved from 0.23984 to 0.22907, saving model to Test1.h5
Epoch 4/150
598/598 [==============================] - 1s 2

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Accuracy: 0.7047863796980405
Recall: 0.6242053789731051
Specificity: 0.6766498807315133


#### Feature Reduction for ANN Classifier

Following Model Has 13 features, reduced from 49 and 42 earlier

In [ ]:
myData_reduced_13 = myData.drop(["Number_of_Patients_In_Waiting_Before_CTAS",
                        "Number_of_Patients_In_Waiting_Before",
                        "ARRIVAL_MODE_Air Ambulance",
                        "PTN_AGE_>100",
                        "PTN_AGE_1-10",
                        "Labs_Ordered_Hour_After_Arrival",
                        "INIT_LOC_GRP_WR Non ED",
                        "Labs_Ordered_Hour_Before_Arrival",
                        "INIT_LOC_GRP_Intake",
                        "INIT_LOC_GRP_MET",
                        "INIT_LOC_GRP_Main",
                        "DIs_Ordered_Hour_Before_Arrival",
                        "DIs_Ordered_Hour_After_Arrival",
                        "PTN_AGE_91-100",
                        "time_of_day_of_arrival_Morning Dawn",
                        "time_of_day_of_arrival_Morning Dusk",
                        "time_of_day_of_arrival_Midday",
                        "time_of_day_of_arrival_Night/Evening",
                        "PTN_AGE_11-20",
                        "INIT_LOC_GRP_WR MET",
                        "ARRIVAL_MODE_Ground Ambulance",
                        "PTN_AGE_81-90",
                        "PTN_AGE_71-80",
                        "ARRIVAL_MODE_No Ambulance",
                        "INIT_TREAT_LOC_GRP_Main",
                        "PTN_AGE_61-70",
                        "PTN_AGE_51-60",
                        "PTN_AGE_41-50",
                        "INIT_LOC_GRP_WR Intake",
                        "PTN_AGE_21-30",
                        "PTN_AGE_31-40",
                        "INIT_LOC_GRP_WR Main",
                        "INIT_TREAT_LOC_GRP_Intake",
                        "INIT_TREAT_LOC_GRP_MET",
                        "PTN_SEX_F",
                        "PTN_SEX_M"],axis=1)
Xtrain_13, Xtest_13, yTrain_13, yTest_13 = splitIntoTrainTest(myData_reduced_13)

In [ ]:
model_13 = buildModel(13)
fitModel(myData_reduced_13, model_13, Xtrain_13, yTrain_13, Xtrain_13, yTrain_13)

Epoch 1/150
598/598 [==============================] - 3s 3ms/step - loss: 0.2494 - accuracy: 0.4960 - precision_8: 0.4322 - recall_8: 0.7160 - val_loss: 0.2490 - val_accuracy: 0.5529 - val_precision_8: 0.4716 - val_recall_8: 0.7630

Epoch 00001: val_loss improved from inf to 0.24899, saving model to Test1.h5
Epoch 2/150
598/598 [==============================] - 1s 2ms/step - loss: 0.2473 - accuracy: 0.5666 - precision_8: 0.4814 - recall_8: 0.7237 - val_loss: 0.2440 - val_accuracy: 0.6005 - val_precision_8: 0.5087 - val_recall_8: 0.7077

Epoch 00002: val_loss improved from 0.24899 to 0.24404, saving model to Test1.h5
Epoch 3/150
598/598 [==============================] - 1s 2ms/step - loss: 0.2416 - accuracy: 0.5973 - precision_8: 0.5064 - recall_8: 0.6994 - val_loss: 0.2379 - val_accuracy: 0.6049 - val_precision_8: 0.5125 - val_recall_8: 0.7153

Epoch 00003: val_loss improved from 0.24404 to 0.23794, saving model to Test1.h5
Epoch 4/150
598/598 [==============================] - 1s 2

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Accuracy: 0.6932426643820947
Recall: 0.5982332155477031
Specificity: 0.7720521172638436
